In [2]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import *

import keyword

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math

# LN7 : 주가수익률 분포(2)
## 2 주가등락을 이항분포로 설명
- 주가등락은 매일은 베르누이 시행, 누적되면 이항분포
- 이항분포의 성공확률 p를 알기 위해 과거 데이터를 사용한다

In [8]:
# 데이터를 불러올 때
# date를 인덱스 칼럼으로 지정하고, index_col='Date'
# 그 칼럼이 날짜로 되어있는걸 알려주면서 하자, parse_dates=True

df=pd.read_csv('c:/temp/samsung_LN7.csv',index_col='Date', parse_dates=True)
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2010-01-04,16060,16180,16000,16180,239271,0.012516
2010-01-05,16520,16580,16300,16440,559219,0.016069
2010-01-06,16580,16820,16520,16820,459755,0.023114
2010-01-07,16820,16820,16260,16260,443237,-0.033294
2010-01-08,16400,16420,16120,16420,295798,0.009840


In [9]:
# 최근 1년치만 가지고 p를 계산해보자
# 2021-01-01: 2021-12-31
# loc: label을 이용해서 가져오는 방식
df1=df.loc['2021-01-01':'2021-12-31']
df1.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691
2021-01-05,81600,83900,81600,83900,35335669,0.010843
2021-01-06,83300,84500,82100,82200,42089013,-0.020262
2021-01-07,82800,84200,82700,82900,32644642,0.008516
2021-01-08,83300,90000,83000,88800,59013307,0.071170


In [12]:
# return 수익률 r
# 양수면 주가 상승, 음수면 주가하락
# python에서 계산 가능

df1['ret']=100*(df1['Close'].pct_change())
df1.head()

,Open,High,Low,Close,Volume,Change,ret
Date,,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691,NaN
2021-01-05,81600,83900,81600,83900,35335669,0.010843,1.084337
2021-01-06,83300,84500,82100,82200,42089013,-0.020262,-2.026222
2021-01-07,82800,84200,82700,82900,32644642,0.008516,0.851582
2021-01-08,83300,90000,83000,88800,59013307,0.071170,7.117008


In [13]:
# ret 변수 만들어짐
# 2021-01-04는 결측치일 수 밖에 없음, 앞에 데이터가 없으니까

In [14]:
df2=df1['ret'] # return 만 있는 데이터 프레임
df2

Date
2021-01-04         NaN
2021-01-05    1.084337
2021-01-06   -2.026222
2021-01-07    0.851582
2021-01-08    7.117008
                ...   
2021-12-24    0.750939
2021-12-27   -0.372671
2021-12-28    0.124688
2021-12-29   -1.867995
2021-12-30   -0.634518
Name: ret, Length: 248, dtype: float64

In [15]:
r=df2.dropna() # dropna(): 결측치만 없애겠다
r

Date
2021-01-05    1.084337
2021-01-06   -2.026222
2021-01-07    0.851582
2021-01-08    7.117008
2021-01-11    2.477477
                ...   
2021-12-24    0.750939
2021-12-27   -0.372671
2021-12-28    0.124688
2021-12-29   -1.867995
2021-12-30   -0.634518
Name: ret, Length: 247, dtype: float64

In [16]:
# r: 필요한 data가 들어있는 데이터 프레임이다!!
len(r) # r의 길이 = 2022년의 영업일

247

In [17]:
# r에 247일 동안 주가가 오르고 내린게 기록되어 있는 것

r.describe()

count    247.000000
mean      -0.014333
std        1.369006
min       -3.496503
25%       -0.731262
50%       -0.123305
75%        0.724664
max        7.117008
Name: ret, dtype: float64

In [20]:
# 2022년 평균적으로는 -0.01%,,, 거의 평균 수익률은 0

# 알고 싶은 것: 247일 동안 몇 일 동안 + 인가
# + 인게 몇 개인지 계산하기
positive_count=(r>0).sum()
positive_count

110

In [21]:
# 2021년 1년동안 110일만큼은 주가가 증가함

In [22]:
110/247 # 주가가 오를 확률

0.44534412955465585

In [24]:
# p = 0.4453
# 5일간 삼전 보유할 예정, 그중에서 3일이 주가가 상승할 확률은 얼마인가?
prob=math.comb(5,3)*(0.4453**3)*(1-0.4453)**2
prob

0.2716904775542357

In [25]:
# 27%

## 3 합의 표준오차

- 이항분포는 정규분포로 근사 가능
- 정규분포 이용해서 확률 구할 수 있음
- 정규분포의 평균, 표준오차를 알아야 함

### 평균
- 필참
### 합의 표준오차
- 필참
- 손계산 or python으로 계산
- 합(며칠이 오를꺼냐)

In [27]:
# 합의 표준오차 손 계산
# 먼저 상자의 분산 varX 계산
varX=0.4453-(0.4453**2)
sdX=np.sqrt(varX)
sdX # 상자의 표준편차

0.4969989034193134

In [29]:
# 합의 표준오차 = 상자의 표준편차*sqrt(시행횟수)
se=sdX*np.sqrt(100)
se

4.969989034193134

In [31]:
# 우리에게 필요한 합의 정규분포: 평균 44.53, 표준오차가 4.969인 분포가 된다
# 평균적으로 44일 정도 오르는데 +-5일정도 오차가 있다..는 뜻

## 4 비율의 표준오차

- 100일 동안 보유할껀데, 주가가 100일 중에 주가가 오르는 날의 비율이 몇 퍼센트야?
- 247일간 보유할건데, 이 중에 주가가 오를 확률은 44.5%(아까꼐산한거) 임. 
- 비율에 대한 평균과 표준오차 계산하는 것

### 평균
- 필참

In [32]:
# 비율의 표준오차를 계산한다.
# 삼성전자 주식을 247일(1년)동안 보유한다고 가정한다.
# 주가가 오른 날의 비율 ~ 평균=0.4453, 표준오차?

#표준오차
se_ratio=sdX/np.sqrt(247)
print(se_ratio)

0.03162328294613793


In [33]:
# 평균적으로는 44.53%라고 오르는 비율이지만, +- 3.1% 정도는 왔다갔다 한다.

## 5 최대 예상손실액 구하기

- 최대예상손실액 = VaR = Value at Risk
- VaR: 위험관리를 위해 만들어진 개념, 정상적인 시장 하에서 일정기간동안 발새알 수 있는 최대손실금액을 의미
- 95% VaR=1,000만원 : 해당 자산의 가치변동으로 인해 6개월 동안 발생가능할 수 있는 최대 손실금액이 1000만원을 넘지 않을 확률이 95% = 손실금액이 1000만원 초과할 확률이 5%
- 근데 5%의 확률로 손실하는 1000만원이라는 값!! 은 어케 나온거임?
- 근거가 있어야함. 과거의 데이터에서 온 것
- 삼전 과거 10년치를 봤음, 봤더니 극단적인 상황이 나올 가능성이 가령 5%의 가능성으로 극단적인 -나오는게 어느정도인지 보는 것
- 왼쪽 꼬리 가 5%정도인 -수익률이 얼마냐라는 것

> 구하는 방법
1) bootstrapping
